In [1]:
%pip install llama-index>=0.11.20
%pip install pymupdf4llm>=0.0.17
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


c:\Users\n.fretti\Desktop\projects\rag_and_roll\.venv\Scripts\python.exe: No module named pip


Note: you may need to restart the kernel to use updated packages.


c:\Users\n.fretti\Desktop\projects\rag_and_roll\.venv\Scripts\python.exe: No module named pip


Note: you may need to restart the kernel to use updated packages.


c:\Users\n.fretti\Desktop\projects\rag_and_roll\.venv\Scripts\python.exe: No module named pip


In [2]:
!mkdir -p 'data/'
!wget 'https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/query_engine/pdf_tables/billionaires_page.pdf?raw=true' -O 'data/billionaires_page.pdf'

The syntax of the command is incorrect.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from rich import print as rprint
import os

In [ ]:
# set the OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = "here your openai api key"

True

In [6]:
Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [7]:
import pymupdf4llm
import pathlib as pl
md_text = pymupdf4llm.to_markdown("./data/billionaires_page.pdf")
pl.Path("data/output.md").write_bytes(md_text.encode())

Processing ./data/billionaires_page.pdf...
[                                        ] (0/3[=                                       ] ( 1/3[==                                      ] ( 2/33[===                                     ] ( 3/3[====                                    ] ( 4/33=[======                                  ] ( 5/33[=======                                 ] ( 6/3[========                                ] ( 7/33[=========                               ] ( 8/3[==========                              ] ( 9/33=[============                            ] (10/33[=============                           ] (11/3[==============                          ] (12/33[===============                         ] (13/3[================                        ] (14/33=[==================                      ] (15/33[===================                     ] (16/3[====================                    ] (17/33[=====================                   ] (18/3=[=======================        

118091

In [8]:
from llama_index.core.node_parser import SentenceSplitter, MarkdownNodeParser
from llama_index.core import VectorStoreIndex
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.extractors import (
    KeywordExtractor,
)
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

def create_index(text_path: str) -> VectorStoreIndex:
    pipeline = IngestionPipeline(
        transformations=[
            MarkdownNodeParser(),
            SentenceSplitter(
                chunk_size=512, chunk_overlap=20
            ),
            KeywordExtractor(keywords=3)
        ],
    )
    documents = SimpleDirectoryReader(text_path, required_exts=[".md"]).load_data()
    nodes = pipeline.run(documents=documents)
    index = VectorStoreIndex(nodes=nodes)
    return index

In [9]:
index = create_index("data")
engine = index.as_query_engine()

100%|██████████| 85/85 [00:14<00:00,  5.90it/s]


In [10]:
rprint(engine.query("""Who was the second richest person in the world in 2023?""").response)

Elon Musk was the second richest person in the world in 2023.

In [11]:
rprint(engine.query("""What was the net worth of the richest person in 2023?""").response)

US$211 billion

In [12]:
rprint(engine.query("""What was the age of the richest person in 2022?""").response)

49

In [13]:
rprint(engine.query("""Which was the primary source of wealth of the richest person in 2022?""").response)

The primary source of wealth of the richest person in 2022 was Tesla, SpaceX, and Twitter Inc.